In [4]:
#data                               
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#nlp
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.stem import PorterStemmer
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')
# nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation

# wordcloud
from PIL import Image
from wordcloud import STOPWORDS, WordCloud

## translation
# import googletrans
# from googletrans import Translator

# Preprocessing

In [5]:
# #whole dataframe
# def get_data(artist_id_input):
#     total_df = pd.read_csv('data/indie_final_dataframe')
#     df = total_df[total_df['artist_id']==int(artist_id_input)][['artist_name', 'song_name', 'year', 'lyrics']]
#     return df

# def basic_cleaning(artist_id):  
#     df = get_data(artist_id)
#     df['final_lyrics'] = df['Lyric'].apply(clean_text)
#     df = df.groupby(['year']).sum()[['final_lyrics']]

#     return df

def clean_text(text):
    stopwords_lst = list(set(stopwords.words('english')))
    manual_sw = ['그','내','난','나는','나를','너를','너의','너와','더','내게','내가','oh','amp','im',
                 '이렇게','이제','너는','니가','할','또','네가','na','수','날','아','게','그대','그대는',
                 '널','네','것','더','건','그대의','언제나','다','이제는','그녀의','위해','같은','아','있는',
                 '이','yeah','마','듯','니','youre','ye','ohh','두','eh','사이니까','will','오예','너무',
                 '너랑','ah','좀','woo','자꾸','쉭','u','나의','s','아','네','안','la','으','거야','babe',
                 '아아아아아','t','올','ya','gray','m','하지','돼','la','넌','너','그런','너에게','속에','엉',
                 'woo','inside','부비대','정말','around','수도','u','꼭','두','1','우린','나','있어','너무나',
                 '순','없는','아직도','마치','앞','항상','함께','않아','하지만','막','나에게','정말','다시','바로','싶어',
                 '많은','어떻게','걸','one','없어','그대와','let','없잖어','아무것도','팡','아직','우리','을','를',
                 '계속','아무','모두','모든','그것이','없는데','got','오예','당장','take','다른','나랑은','수많은',
                 'll','해봐','멀리','그저','기덕','못','받거니','늘','않는','사는','주거니','해야','걸','아닌','이고',
                 '못할','da','cha','asunder','이젠','손을','그렇게','해도','baby','해','tickin','수가','하고',
                 'gon','go','say','때','것을','채','단','눈을','예','한','cream','christmas','merry','make',
                 'know','her의','we는','눈이','볼','바바바','너에게ohh','people','give','90s','wan','하고','대로',
                 '줄','잘','알','될','come','acdc','이런','raw','de','que','look''아아아','bye','있을','하는','저',
                 '그렇게','쿵','참','말아라','chest에','님아','red','오오','모십니다','maker','우','어떤','we가',
                 'waterfalls','tell','음악으로','풍문으로','build','don','talk','보여','say','blue','techno',
                 'shoes','와','누가','되어','가는','고개','gim','들어','가지','we는','doo','alway','것만','tell',
                 'bie','춤을','green','채팅','먹어','umm','대로','순간','know','그건','나와','cruise','coastal',
                 'classic','playa','light','하는','see','그래','eye','time','비가','sun','hi','sun','surf',
                 'ive','위에','달이','안엔','hey','지금','우','dive','cause','하나','see','know','에오','길을',
                 '맞닿음','보고','될','음','아주','이유','나도','몇','surf', 'uh', '오',]
    stopwords_lst.extend(manual_sw)
    #changing to lowercase
    text = text.lower()
    text = text.replace('\r',' ')

    # removing #´s 
    text = re.sub(r'#[A-Za-z0-9]+', ' ', text)
    text = re.sub(r'#', ' ', text)

    #indentation -> space
    text = re.sub(r'\n', ' ', text)

    for punctuation in string.punctuation:
        text = text.replace(punctuation, '') 

    #strip
    text = text.strip()

    #tokenization
    tokens = word_tokenize(text)   
    filtered_words = [w for w in tokens if w not in stopwords_lst]
    final = " ".join(cat for cat in filtered_words)

    return final


In [6]:
df = pd.read_csv('data/indie_final_df.csv', index_col=0)

In [7]:
df['final_lyrics'] = df['lyrics'].apply(clean_text)
df

,artist_id,artist_name,song_id,song_name,album_id,album,like_count,lyrics,date,year,final_lyrics
0,31236,휘성(Realslow),2275648,가슴 시린 이야기 (feat. 용준형 Of Beast),677972,가슴 시린 이야기,"[14,786]",[세상에 가장 예쁜 거짓말 goodbye goodbye\r\n가슴 찢겨도 뱉는 그 ...,2011-03-15,2011,세상에 가장 예쁜 거짓말 goodbye goodbye 가슴 찢겨도 뱉는 말 good...
1,31236,휘성(Realslow),2063982,결혼까지 생각했어,270770,Realslow Is Back,"[18,843]",[To all My people hellow hellow hellow hellow ...,2010-08-26,2010,hellow hellow hellow hellow name realslow call...
2,31236,휘성(Realslow),2063984,사랑 그 몹쓸 병,270770,Realslow Is Back,"[2,787]",[곁에 오지 말아요\r\n나를 보지 말아요\r\n단 한번 단 한번의 호흡조차 위험해...,2010-08-26,2010,곁에 오지 말아요 보지 말아요 한번 한번의 호흡조차 위험해 건네오는 인사에 스쳐가는...
3,31236,휘성(Realslow),1700206,Insomnia (불면증),324617,Insomnia (불면증),"[41,145]","[내가 달리는 길은 Love, Love, Love, Love \r\n허나 그 길엔 ...",2009-02-18,2009,달리는 길은 love love love love 허나 길엔 온통 덫 덫 덫 덫 피할...
4,31236,휘성(Realslow),3295630,Special Love,445300,Special Love,"[2,157]",[대화 따위는 필요없어 \r\n서로 눈빛으로 모든 걸 알 수 있어 \r\n뭔가 흐름...,2013-11-29,2013,대화 따위는 필요없어 서로 눈빛으로 뭔가 흐름이 좋아 만남이 주는 기분에 취하고 황...
...,...,...,...,...,...,...,...,...,...,...,...
45,6653,가리온(Garion),30503578,[19금],20022222,금기어,[16],[],2016-12-30,2016,
46,6653,가리온(Garion),3325821,그래서 함께 하는 이유 2013 (inst.),329757,가리온 15주년 기념 앨범 Instrumental,[1],[],2013-12-24,2013,
47,6653,가리온(Garion),31200427,변,20055463,변,[17],[뭐 하냐고? 뭐 하냐고??\r\n(여보세요? 존나 랩해)\r\n가리온 퇴물랩 나이...,2018-08-23,2018,뭐 하냐고 뭐 하냐고 여보세요 존나 랩해 가리온 퇴물랩 나이만 처먹네 음원을 내봐도...
48,6653,가리온(Garion),32050291,그 놈의 음악 (Prod. Jinhu),20036214,그 놈의 음악,[13],[비싼 건 손도 못 대던 시절\r\n먹고 살 수만 있어도 인정\r\n알바는 옵션이 ...,2020-10-28,2020,비싼 손도 대던 시절 먹고 살 수만 있어도 인정 알바는 옵션이 기본 돈만 주면 뭐든...


In [8]:
df_filtered = df[['artist_id','artist_name', 'song_name', 'year', 'final_lyrics']]
df_filtered.head()

,artist_id,artist_name,song_name,year,final_lyrics
0,31236,휘성(Realslow),가슴 시린 이야기 (feat. 용준형 Of Beast),2011,세상에 가장 예쁜 거짓말 goodbye goodbye 가슴 찢겨도 뱉는 말 good...
1,31236,휘성(Realslow),결혼까지 생각했어,2010,hellow hellow hellow hellow name realslow call...
2,31236,휘성(Realslow),사랑 그 몹쓸 병,2010,곁에 오지 말아요 보지 말아요 한번 한번의 호흡조차 위험해 건네오는 인사에 스쳐가는...
3,31236,휘성(Realslow),Insomnia (불면증),2009,달리는 길은 love love love love 허나 길엔 온통 덫 덫 덫 덫 피할...
4,31236,휘성(Realslow),Special Love,2013,대화 따위는 필요없어 서로 눈빛으로 뭔가 흐름이 좋아 만남이 주는 기분에 취하고 황...


In [9]:
df_filtered.to_csv('data/indie_final_df_cleaned.csv')

# tf-idf

In [ ]:
#by_year
df_by_year = df_filtered.groupby('year')[['final_lyrics']].sum()

In [ ]:
df_by_year = df_by_year.loc[2008:]
df_by_year

In [ ]:
vectorizer = TfidfVectorizer(max_features = 2000, ngram_range = (2,2)) #min_df = 0.01, max_df = 0.95,  ngram_range = (2,2)
X = vectorizer.fit_transform(df_by_year['final_lyrics'])
vectorized_ngram = pd.DataFrame(X.toarray(),
columns = vectorizer.get_feature_names_out())

In [ ]:
new_df = vectorized_ngram.transpose()
new_df['sum'] = new_df.sum(axis=1)
new_df = new_df.reset_index()

# keywords = new_df[new_df['index'].str.contains(keyword_)]
result = new_df[['index', 'sum']].sort_values(by='sum', ascending = False)

In [ ]:
result[result['index'].str.contains('없이')]

## vectorizer by year

In [ ]:
df_filtered

In [ ]:
def vectorizer_by_year(yr):
    df_year = df_filtered[df_filtered['year']==yr]
    vectorizer = TfidfVectorizer(min_df = 0.01, max_df = 0.95, max_features = 50) #, ngram_range = (2,2)
    X = vectorizer.fit_transform(df_year['final_lyrics'])
    vectorized_ngram = pd.DataFrame(X.toarray(),
    columns = vectorizer.get_feature_names_out())
    
    new_df = vectorized_ngram.transpose()
    new_df['sum'] = round(new_df.sum(axis=1))
    new_df = new_df.reset_index()

    # keywords = new_df[new_df['index'].str.contains(keyword_)]
    result = new_df[['index', 'sum']].sort_values(by='sum', ascending = False).reset_index(drop=True).rename(columns = {'index': 'word'})
    result['year'] = yr
    
    return result[['year', 'word', 'sum']]

In [ ]:
dataframe_top50_words = []

for yr in range(2008, 2024):
    dataframe_top50_words.append(vectorizer_by_year(yr))

In [ ]:
top50words_df = pd.concat(dataframe_top50_words, axis=0)
top50words_df.to_csv('keywords_by_year_vertically_concat.csv')

# Wordcloud

In [ ]:
df_by_year = df_by_year.loc[2000:]
df_by_year

In [ ]:
def simple_wordcloud_2(total):
    
    wordcloud = WordCloud(font_path = 'LINESeedKR-Rg.otf', width = 1000, height = 500, colormap='RdPu_r', 
                        max_words=150, background_color= 'white', collocations = False).generate(total)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    
#     wordcloud.to_file(f"final_wordcloud_png/{artist_id}_wordcloud.png")
    

In [ ]:
def simple_wordcloud(artist_id):
    df = basic_cleaning(artist_id)
    
    total_lyrics = " ".join(cat for cat in df.final_lyrics)
    
    wordcloud = WordCloud(font_path = 'LINESeedKR-Rg.otf', width = 1000, height = 500, colormap='RdPu_r', 
                        max_words=70, background_color= 'white', collocations = False).generate(total_lyrics)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    
#     wordcloud.to_file(f"final_wordcloud_png/{artist_id}_wordcloud.png")
    

In [ ]:
def wordcloud_by_year(artist_id):
    
    df = basic_cleaning(artist_id)
    
    def by_year(df):
        lyrics_of_years = []
        for i in range(len(df)):
            one_year = " ".join(cat for cat in df.iloc[i, :])
            lyrics_of_years.append(one_year)
        return lyrics_of_years


    lyric_lst = by_year(df)
    year_lst = [i for i in df.index]
    
    for lyric in lyric_lst:
        word_cloud = WordCloud(font_path='Binggrae', width = 1000, height = 500, 
        colormap='BuPu', max_words=100, collocations = False).generate(lyric)

        plt.figure(figsize = (10,8))
        plt.imshow(word_cloud, interpolation='bilinear')
        plt.axis("off")
        plt.title(year_lst[lyric_lst.index(lyric)])
        plt.show()
        
        

## total wordcloud

In [ ]:
indie_total_df = pd.read_csv('data/indie_final_dataframe')
indie_total_df = indie_total_df[['artist_x', 'artist_id', 'song_name', 'year', 'Lyric']]
indie_total_df['final_lyrics'] = indie_total_df.Lyric.apply(clean_text)
indie_total_df.drop(columns = ['Lyric'], inplace = True)
indie_total_df

In [ ]:
total_lyrics = " ".join(cat for cat in indie_total_df.final_lyrics)
# total_lyrics

In [ ]:
len(total_lyrics)

In [ ]:
indie_temp = indie_total_df[['year', 'final_lyrics']]
indie_by_year_df = indie_temp.groupby(['year']).sum()[['final_lyrics']]
indie_by_year_df.dropna(inplace=True)

In [ ]:
indie_by_year_df = indie_by_year_df.loc[2008:]
indie_by_year_df

In [ ]:
def wc_by_year_2(df):
    def by_year(df):
        lyrics_of_years = []
        for i in range(len(df)):
            one_year = " ".join(cat for cat in df.iloc[i, :])
            lyrics_of_years.append(one_year)
        return lyrics_of_years


    lyric_lst = by_year(df)
    year_lst = [i for i in df.index]

    for lyric in lyric_lst:
        word_cloud = WordCloud(font_path='Binggrae', width = 1000, height = 500, 
        colormap='BuPu', max_words=100, collocations = False).generate(lyric)

        plt.figure(figsize = (10,8))
        plt.imshow(word_cloud, interpolation='bilinear')
        plt.axis("off")
        plt.title(year_lst[lyric_lst.index(lyric)])
        plt.show()
        
        yearr=year_lst[lyric_lst.index(lyric)]
        word_cloud.to_file(f"data/final_wordcloud_png/{yearr}_wordcloud.png")

In [ ]:
wc_by_year_2(indie_by_year_df)

# Topic Clustering

In [24]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
import pandas as pd

2023-10-05 18:53:46.171778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [41]:
df = pd.read_csv('data/indie_final_df_cleaned.csv', index_col=0)
df['final_lyrics'] = df.final_lyrics.astype('str')
df = df.final_lyrics.values.tolist()

In [42]:
# # Turn the list of string into a list of tokens
df = [t.split() for t in df]

In [43]:
id2word = Dictionary(df)
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in df]

In [44]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

[[('goodbye', 22),
  ('rap안녕', 1),
  ('가슴', 2),
  ('가장', 1),
  ('거짓말', 4),
  ('결국', 1),
  ('고마웠었어', 1),
  ('괜찮아', 1),
  ('그래도', 2),
  ('그말', 1),
  ('나만', 2),
  ('남자로써', 1),
  ('남자의', 2),
  ('낳은', 1),
  ('내사랑', 3),
  ('누군가에게', 1),
  ('느끼던', 1),
  ('늘어난', 1),
  ('더듬대며', 1),
  ('동안', 1),
  ('둘이서', 1),
  ('드라마', 1),
  ('리가', 1),
  ('마지막', 1),
  ('마지막으로', 1),
  ('만나', 1),
  ('만남', 1),
  ('만든', 1),
  ('만큼', 2),
  ('말', 3),
  ('말도', 1),
  ('말이', 1),
  ('맞추던', 1),
  ('무겁게', 1),
  ('미워도', 3),
  ('미워서', 1),
  ('믿었던', 1),
  ('바보야', 1),
  ('뱉는', 1),
  ('버려진다', 1),
  ('베낀다', 1),
  ('보내는', 2),
  ('보낼게', 1),
  ('부모를', 1),
  ('뺏긴다', 1),
  ('뻔한', 1),
  ('사람', 4),
  ('사랑', 2),
  ('사랑은', 1),
  ('사랑을', 3),
  ('사랑한', 1),
  ('살아', 1),
  ('세상에', 1),
  ('속', 1),
  ('슬퍼서', 1),
  ('시린', 1),
  ('아이처럼', 1),
  ('아파', 4),
  ('악물고', 1),
  ('안고서', 1),
  ('안녕', 3),
  ('안녕이란', 1),
  ('안녕이지', 1),
  ('안되는', 1),
  ('알려준', 3),
  ('앞에', 1),
  ('어떻게든', 2),
  ('없잖아', 1),
  ('예쁜', 1),
  ('우리가', 1),
  ('울지마', 1),
  ('위한', 1),
 

In [46]:
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=20,
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.035*"아니야" + 0.032*"어쩌면" + 0.032*"아니" + 0.028*"겨우" + 0.028*"그때" + '
  '0.026*"말이야" + 0.025*"지나" + 0.021*"어" + 0.019*"안녕" + 0.019*"이야기"'),
 (1,
  '0.039*"마음은" + 0.033*"지난" + 0.030*"걸음" + 0.024*"앉아" + 0.020*"꿈은" + '
  '0.016*"버려" + 0.015*"집으로" + 0.015*"이유를" + 0.015*"좋아요" + 0.014*"돌아온"'),
 (2,
  '0.036*"있다" + 0.034*"같이" + 0.029*"위" + 0.025*"모두가" + 0.021*"미친" + '
  '0.017*"wait" + 0.015*"남겨진" + 0.014*"없인" + 0.014*"몸에" + 0.013*"걷던"'),
 (3,
  '0.072*"그래서" + 0.040*"때까지" + 0.036*"가끔" + 0.019*"사람들이" + 0.010*"안아줘" + '
  '0.009*"말들" + 0.009*"시작하는" + 0.007*"나갈" + 0.006*"살이" + 0.005*"안아줄"'),
 (4,
  '0.042*"오직" + 0.037*"소리" + 0.036*"꿈을" + 0.025*"높이" + 0.023*"향해" + 0.018*"아래" '
  '+ 0.018*"마음을" + 0.017*"바람" + 0.017*"않고" + 0.016*"생각이"'),
 (5,
  '0.050*"’" + 0.049*"dont" + 0.031*"never" + 0.031*"like" + 0.028*"feel" + '
  '0.027*"ill" + 0.024*"away" + 0.020*"day" + 0.019*"get" + 0.018*"back"'),
 (6,
  '0.050*"그냥" + 0.030*"혼자" + 0.027*"그게" + 0.027*"가" + 0.027*"깊은" + 0.019*"먼저" '
  '+ 0.016*"차라리

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df_filtered.final_lyrics.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]